In [1]:
# import libraries
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from unityagents import UnityEnvironment

ModuleNotFoundError: No module named 'unityagents'

In [ ]:
# import libraries
import agent
import config

In [ ]:
# init
os.makedirs(config.SAVE_DIRECTORY, exist_ok=True)

In [ ]:
# load game environment
env = UnityEnvironment(file_name=config.UNITY_ENVIRONMENT_PATH, no_graphics=config.UNITY_NO_GRAPHIC, worker_id=config.UNITY_WORKER_ID))
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=True)[brain_name]

In [ ]:
# extract game configuration
num_agents = len(env_info.agents)
states = env_info.vector_observations
input_dims = states.shape[1]
action_dims = brain.vector_action_space_size

In [ ]:
# initialise agent
agent = agent.DDPG(input_dims, action_dims, num_agents)

In [ ]:
# create training function
def train(n_episodes, num_agents, score_buffer=100):

    # init
    scores = []
    scores_window = deque(maxlen=score_buffer)

    # start training
    for eps in range(n_episodes):

        # re-init
        agent.reset()
        score = np.zeros(num_agents)

        # reset env
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        
        # iterating within single episode
        while True:

            # get optimal actions based on latest agent policy
            actions = agent.act(states)
        
            # execute actions
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done

            # learn from actions
            agent.step(states, actions, rewards, next_states, dones)

            # store rewards
            score += rewards
        
            # stop iteration if it reach terminal                                                        
            if np.any(dones):   
                break                                        

            # setting preparation for next iteration
            states = next_states       

        # save score into log and buffer
        scores.append(np.max(score))
        scores_window.append(np.max(score))
        print("episode: {}/{}   score: {}   average score: {}".format(eps, n_episodes, np.max(score), np.mean(scores_window)), end="\r")
        
        # stop if environment is solved
        if np.mean(scores_window) >= 30:
            print("environment solved in {} episode with average score = {}".format(eps, np.mean(scores_window)))
            torch.save(agent.actor_local.state_dict(), '{}/actor_model.pth'.format(config.SAVE_DIRECTORY))
            torch.save(agent.critic_local.state_dict(), '{}/critic_model.pth'.format(config.SAVE_DIRECTORY))
            break
    
    return scores

# start training
scores_log = train(config.N_EPISODES, num_agents)

In [ ]:
# plot results
x = np.arange(1, len(scores_log)+1)
y = scores_log
plt.plot(x, y)
plt.title("Continuous Control Episode vs Rewards")
plt.xlabel("Episode")
plt.ylabel("Average Rewards")
plt.show()